In [1]:
dataset_directory = '/Users/chris/Library/CloudStorage/GoogleDrive-cjameswalmsley@gmail.com/My Drive/Shared with Julia/Education/Kent University/MSc/Modules/COMP8260_AI_Systems_Implementation/Project/dataset/archive'

In [8]:
from os import path, listdir
from cv2 import imread
import numpy as np

In [5]:
def load_image(file_path):
    return imread(file_path)

In [10]:
#load training images
#img = cv2.imread(os.path.join(dataset_directory, 'train/images/0001.jpg'))
#images = imread(path.join(dataset_directory, 'train/images/0001.jpg'))
#x = np.array([np.array(Image.open(fname)) for fname in filelist])

directory_path = path.join(dataset_directory, 'train/images')

for file in listdir(directory_path):
    if file.endswith('.jpg'):
        img = load_image(path.join(directory_path, file))
        print(img.shape)
        break


(512, 420, 3)
